In [1]:
%load_ext lab_black

In [2]:
import os

os.chdir("../..")

In [3]:
import pandas as pd
import numpy as np
import swifter
from gensim.corpora.dictionary import Dictionary
from gensim.models.ldamulticore import LdaMulticore
from utils.cluster_summary import get_cluster_summary

pd.options.display.float_format = "{:,.2f}".format

In [4]:
train_df, test_df = (
    pd.read_pickle("resources/data/train_data.pkl"),
    pd.read_pickle("resources/data/test_data.pkl"),
)

In [5]:
lda_topics_num = 4

In [6]:
def bow_to_list(bow):
    l = [[item[0]] * item[1] for item in bow.items()]
    l = [str(item) for sublist in l for item in sublist]
    return l

In [7]:
train_token_lists = train_df["bow_dict"].swifter.apply(bow_to_list)
test_token_lists = test_df["bow_dict"].swifter.apply(bow_to_list)

Pandas Apply:   0%|          | 0/5376 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/2304 [00:00<?, ?it/s]

In [8]:
dictionary = Dictionary(train_token_lists)
train_bows = train_token_lists.swifter.apply(dictionary.doc2bow)
test_bows = test_token_lists.swifter.apply(dictionary.doc2bow)

lda = LdaMulticore(train_bows, num_topics=lda_topics_num)

Pandas Apply:   0%|          | 0/5376 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/2304 [00:00<?, ?it/s]

In [9]:
def get_lda_topics(bows, lda) -> pd.DataFrame:
    pred = np.array([max(lda[bow], key=lambda topic: topic[1])[0] for bow in bows])
    return pred

In [10]:
train_df = train_df.assign(lda=get_lda_topics(train_bows, lda))
test_df = test_df.assign(lda=get_lda_topics(test_bows, lda))

In [11]:
def get_lda_summary(df):
    summary = pd.pivot(
        data=df.groupby(["label", "lda"], as_index=False).size(),
        index="label",
        columns="lda",
    )
    return summary.div(summary.values.sum(axis=0), axis=1)

In [12]:
get_lda_summary(train_df)

size               
lda        0    1    2    3
label                      
enron   0.13 0.24 0.10 0.27
kos     0.07 0.34 0.04 0.24
nips    0.32 0.04 0.51 0.07
nytimes 0.25 0.24 0.04 0.23
pubmed  0.23 0.14 0.31 0.19

In [13]:
get_lda_summary(test_df)

size               
lda        0    1    2    3
label                      
enron   0.09 0.27 0.03 0.31
kos     0.05 0.33 0.02 0.23
nips    0.38 0.01 0.55 0.05
nytimes 0.26 0.26 0.00 0.22
pubmed  0.22 0.13 0.40 0.19